In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Do new simulations 

In [ ]:
from simulation import run_simulation

### Option 1: Reproduce results

#key = 'fixed_complexity'
#paramfolder = 'results/{}/'.format(key)
#results = run_simulation(paramfolder, outfolder=None)

### Option 2: Add new results to existing folder 

#run_simulation(paramfolder, outfolder=paramfolder)

### Option 3: Created completely new results or add new to existing folder.

parameters = {
    'key': 'fixed_complexity', 
    'n_its': 2, 
    'time': 'undefined', 
    'positions': [15],
    'complexities': [6], 
    'anchors': [10], 
}
outfolder = 'results/{}/'.format(parameters['key'])
run_simulation(parameters, outfolder=None)

# Read and plot results

In [ ]:
from simulation import read_results, read_params
from global_variables import DIM
key =  'fixed_complexity'
resultfolder = 'results/{}/'.format(key)
results = read_results(resultfolder + 'result_')
parameters = read_params(resultfolder + 'parameters.json')

def add_plot_decoration():
    plt.colorbar()
    plt.xlabel('number of missing measurements')
    plt.ylabel('n_positions')
    plt.yticks(range(len(positions)), positions)
    plt.gca().xaxis.tick_bottom()
    plt.show()
    
print("NOTE: values are nan if never visited (white)")
    
if results['successes'].ndim >= 2:
    n_complexity = parameters['complexities'][0]
    positions = parameters['positions']
    n_anchors = parameters['anchors'][0]
    
    plt.matshow(results['successes'])
    plt.title('Number of successful solutions')
    add_plot_decoration()

    plt.matshow(results['num-not-solved'])
    plt.title('Number of solver failures')
    add_plot_decoration()

    plt.matshow(results['num-not-accurate'])
    plt.title('Number of inaccurate solutions')
    add_plot_decoration()

    bound_theory = np.zeros_like(results['successes'])
    n_min = n_complexity * (n_complexity + 1) / 2 + DIM * n_complexity
    print('needed number of measurements:',  n_min)
    for i, n_pos in enumerate(positions):
        max_missing_allowed = max(int(n_pos * n_anchors - n_min), 0)
        if max_missing_allowed < bound_theory.shape[1]:
            bound_theory[i, max_missing_allowed] = 1.0
        if n_pos * n_anchors < bound_theory.shape[1]:
            bound_theory[i, (n_pos * n_anchors):] = np.nan

    print("NOTE: shows theoretical bound () and value from which zero measurements are left (white)")
    plt.matshow(bound_theory)
    plt.title('theoretical bound')
    add_plot_decoration()
else:
    plt.plot(results['successes'], label='success', linestyle="-")
    plt.plot(results['num-not-solved'], label='not-solved', linestyle=":")
    plt.plot(results['num-not-accurate'], label='not-accurate', linestyle=":")
    plt.legend(loc='center left')
    print(str(parameters))
    plt.title('K={}, N={}, M={}'.format(parameters['complexities'][0],
                                        parameters['positions'][0],
                                        parameters['anchors'][0]))
    fig = plt.gcf()
    fig.set_size_inches(10, 5)